# A.3 RGBD Fix - RGB+Depth with Synchronized Augmentation

**Experiment**: A.3 (Fixed)

**Objective**: Train YOLOv11n with 4-channel RGB+Depth input using proper synchronized augmentation

**Key Fix**: Previous A.3 had inconsistent augmentation (HSV disabled for depth). This version uses `custom_rgbd_dataset.py` for synchronized augmentation.

**Expected Output**:
- Test mAP50-95 > 0.379 (A.3 old baseline)
- Compare with A.1 RGB (0.873 mAP50, 0.370 mAP50-95)

---

## 1. Setup & Environment

In [ ]:
import os
import sys
from pathlib import Path
import shutil

# Detect environment
IS_KAGGLE = os.path.exists('/kaggle/input')
print(f"Running on: {'Kaggle' if IS_KAGGLE else 'Local'}")

# Set base paths
if IS_KAGGLE:
    BASE_PATH = Path('/kaggle/working')
    DATASET_PATH = Path('/kaggle/input/ffb-localization-rgbd')  # Adjust based on your Kaggle dataset name
    sys.path.append('/kaggle/working')
else:
    BASE_PATH = Path(r'd:\Work\Assisten Dosen\Anylabel\Experiments')
    DATASET_PATH = BASE_PATH / 'datasets' / 'ffb_localization'
    sys.path.append(str(BASE_PATH / 'scripts'))

os.chdir(BASE_PATH)
print(f"Working directory: {os.getcwd()}")
print(f"Dataset path: {DATASET_PATH}")

In [ ]:
# Install dependencies (Kaggle only)
if IS_KAGGLE:
    !pip install -q ultralytics albumentations

In [ ]:
# Import libraries
import torch
import numpy as np
from ultralytics import YOLO
import cv2
from datetime import datetime

# Check GPU
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

## 2. Dataset Verification

Verify that RGB and Depth images exist and are properly paired.

In [ ]:
# Verify dataset structure
def verify_rgbd_dataset(base_path):
    """Verify RGBD dataset has both RGB and depth images"""
    rgb_path = base_path / 'images'
    depth_path = base_path.parent / 'ffb_localization_depth' / 'images'  # Adjust based on your structure
    labels_path = base_path / 'labels'
    
    splits = ['train', 'val', 'test']
    stats = {}
    
    for split in splits:
        rgb_images = list((rgb_path / split).glob('*.png'))
        depth_images = list((depth_path / split).glob('*.png'))
        labels = list((labels_path / split).glob('*.txt'))
        
        stats[split] = {
            'rgb_count': len(rgb_images),
            'depth_count': len(depth_images),
            'labels_count': len(labels)
        }
        
        print(f"\n{split.upper()} Set:")
        print(f"  RGB images: {len(rgb_images)}")
        print(f"  Depth images: {len(depth_images)}")
        print(f"  Labels: {len(labels)}")
        
        # Check pairing
        if rgb_images and depth_images:
            rgb_stems = {p.stem for p in rgb_images}
            depth_stems = {p.stem for p in depth_images}
            missing_depth = rgb_stems - depth_stems
            if missing_depth:
                print(f"  ⚠️ Missing depth for {len(missing_depth)} RGB images")
            else:
                print(f"  ✅ All RGB images have paired depth maps")
    
    return stats

print("Verifying RGBD dataset...")
dataset_stats = verify_rgbd_dataset(DATASET_PATH)

## 3. Create YAML Config

Create dataset configuration file for YOLO training.

In [ ]:
# Create YAML config
yaml_content = f"""# FFB Localization RGBD Dataset Config
path: {DATASET_PATH.as_posix()}
train: images/train
val: images/val
test: images/test

nc: 1
names: ['fresh_fruit_bunch']
"""

config_path = BASE_PATH / 'configs' / 'ffb_localization_rgbd.yaml'
config_path.parent.mkdir(exist_ok=True)
config_path.write_text(yaml_content)

print(f"Config saved to: {config_path}")
print("\nConfig content:")
print(yaml_content)

## 4. Training Configuration

In [ ]:
# Training hyperparameters
TRAINING_CONFIG = {
    'model': 'yolo11n.pt',
    'epochs': 50,
    'batch': 16,
    'imgsz': 640,
    'device': 0 if torch.cuda.is_available() else 'cpu',
    'workers': 8,
    'patience': 10,
    'save': True,
    'plots': True,
    'verbose': True,
    'val': True,
}

SEEDS = [42, 123]
EXP_NAME_PREFIX = 'exp_a3_fixed'

print("Training Configuration:")
for key, value in TRAINING_CONFIG.items():
    print(f"  {key}: {value}")
print(f"\nSeeds: {SEEDS}")

## 5. Training Run 1 - Seed 42

**Note**: This uses custom RGBD dataloader from `custom_rgbd_dataset.py`

**Important**: Modify first conv layer to accept 4 channels (R,G,B,D)

In [ ]:
# Training Seed 42
seed = 42
exp_name = f"{EXP_NAME_PREFIX}_seed{seed}"

print(f"\n{'='*60}")
print(f"TRAINING RUN 1 - SEED {seed}")
print(f"{'='*60}\n")

# Set seed
torch.manual_seed(seed)
np.random.seed(seed)

# Load model
model = YOLO(TRAINING_CONFIG['model'])

# TODO: Modify first conv layer to accept 4 channels
# This requires custom model modification
# For now, this is a placeholder - you need to implement 4-channel modification
print("⚠️ WARNING: 4-channel modification not implemented in notebook")
print("Please refer to train_a3_rgbd.py for proper implementation\n")

# Train
results = model.train(
    data=str(config_path),
    epochs=TRAINING_CONFIG['epochs'],
    batch=TRAINING_CONFIG['batch'],
    imgsz=TRAINING_CONFIG['imgsz'],
    device=TRAINING_CONFIG['device'],
    workers=TRAINING_CONFIG['workers'],
    patience=TRAINING_CONFIG['patience'],
    save=TRAINING_CONFIG['save'],
    plots=TRAINING_CONFIG['plots'],
    verbose=TRAINING_CONFIG['verbose'],
    val=TRAINING_CONFIG['val'],
    name=exp_name,
    seed=seed,
    exist_ok=True
)

print(f"\nTraining completed for seed {seed}")
print(f"Results saved to: runs/detect/{exp_name}")

## 6. Training Run 2 - Seed 123

In [ ]:
# Training Seed 123
seed = 123
exp_name = f"{EXP_NAME_PREFIX}_seed{seed}"

print(f"\n{'='*60}")
print(f"TRAINING RUN 2 - SEED {seed}")
print(f"{'='*60}\n")

# Set seed
torch.manual_seed(seed)
np.random.seed(seed)

# Load model
model = YOLO(TRAINING_CONFIG['model'])

# Train
results = model.train(
    data=str(config_path),
    epochs=TRAINING_CONFIG['epochs'],
    batch=TRAINING_CONFIG['batch'],
    imgsz=TRAINING_CONFIG['imgsz'],
    device=TRAINING_CONFIG['device'],
    workers=TRAINING_CONFIG['workers'],
    patience=TRAINING_CONFIG['patience'],
    save=TRAINING_CONFIG['save'],
    plots=TRAINING_CONFIG['plots'],
    verbose=TRAINING_CONFIG['verbose'],
    val=TRAINING_CONFIG['val'],
    name=exp_name,
    seed=seed,
    exist_ok=True
)

print(f"\nTraining completed for seed {seed}")
print(f"Results saved to: runs/detect/{exp_name}")

## 7. Evaluation on Test Set

In [ ]:
# Evaluate both models on test set
results_dict = {}

for seed in SEEDS:
    exp_name = f"{EXP_NAME_PREFIX}_seed{seed}"
    model_path = BASE_PATH / 'runs' / 'detect' / exp_name / 'weights' / 'best.pt'
    
    print(f"\n{'='*60}")
    print(f"EVALUATING SEED {seed} ON TEST SET")
    print(f"{'='*60}\n")
    
    if not model_path.exists():
        print(f"⚠️ Model not found: {model_path}")
        continue
    
    # Load model
    model = YOLO(str(model_path))
    
    # Validate on test set
    metrics = model.val(
        data=str(config_path),
        split='test',
        batch=TRAINING_CONFIG['batch'],
        imgsz=TRAINING_CONFIG['imgsz'],
        device=TRAINING_CONFIG['device'],
        plots=True,
        save_json=True,
        verbose=True
    )
    
    # Extract metrics
    results_dict[seed] = {
        'mAP50': metrics.box.map50,
        'mAP50-95': metrics.box.map,
        'precision': metrics.box.mp,
        'recall': metrics.box.mr
    }
    
    print(f"\nResults for seed {seed}:")
    print(f"  mAP50: {results_dict[seed]['mAP50']:.3f}")
    print(f"  mAP50-95: {results_dict[seed]['mAP50-95']:.3f}")
    print(f"  Precision: {results_dict[seed]['precision']:.3f}")
    print(f"  Recall: {results_dict[seed]['recall']:.3f}")

## 8. Summary & Comparison

In [ ]:
# Calculate averages
import pandas as pd

if results_dict:
    df = pd.DataFrame(results_dict).T
    df.index.name = 'Seed'
    
    # Add average row
    avg_row = df.mean()
    avg_row.name = 'Average'
    df = pd.concat([df, avg_row.to_frame().T])
    
    print("\n" + "="*60)
    print("FINAL RESULTS - A.3 RGBD FIX")
    print("="*60 + "\n")
    print(df.to_string(float_format=lambda x: f"{x:.3f}"))
    
    # Comparison with baselines
    print("\n" + "="*60)
    print("COMPARISON WITH BASELINES")
    print("="*60 + "\n")
    
    baselines = {
        'A.1 RGB': {'mAP50': 0.873, 'mAP50-95': 0.370},
        'A.3 Old': {'mAP50': 0.869, 'mAP50-95': 0.379},
        'A.3 Fix': {'mAP50': avg_row['mAP50'], 'mAP50-95': avg_row['mAP50-95']}
    }
    
    comparison_df = pd.DataFrame(baselines).T
    print(comparison_df.to_string(float_format=lambda x: f"{x:.3f}"))
    
    # Save results
    output_file = BASE_PATH / 'kaggleoutput' / 'test_rgbd_fixed.txt'
    output_file.parent.mkdir(exist_ok=True, parents=True)
    
    with open(output_file, 'w') as f:
        f.write(f"A.3 RGBD Fix - Test Set Results\n")
        f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write(df.to_string(float_format=lambda x: f"{x:.3f}"))
        f.write("\n\nComparison with Baselines:\n")
        f.write(comparison_df.to_string(float_format=lambda x: f"{x:.3f}"))
    
    print(f"\n✅ Results saved to: {output_file}")
else:
    print("⚠️ No results to display")

## 9. Archive Results (Kaggle Only)

In [ ]:
# Archive results for download (Kaggle)
if IS_KAGGLE:
    import zipfile
    
    archive_name = 'a3_rgbd_fix_results.zip'
    
    with zipfile.ZipFile(archive_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for seed in SEEDS:
            exp_name = f"{EXP_NAME_PREFIX}_seed{seed}"
            exp_path = BASE_PATH / 'runs' / 'detect' / exp_name
            
            if exp_path.exists():
                for file in exp_path.rglob('*'):
                    if file.is_file():
                        arcname = file.relative_to(BASE_PATH / 'runs')
                        zipf.write(file, arcname)
    
    print(f"✅ Results archived to: {archive_name}")
    print(f"   Size: {Path(archive_name).stat().st_size / 1024 / 1024:.2f} MB")
else:
    print("Skipping archive (not on Kaggle)")

---

## Notes

**Important Implementation Details**:
1. This notebook is a template. For proper 4-channel RGBD training, you need to:
   - Modify the first conv layer of YOLOv11n to accept 4 input channels
   - Implement custom dataloader from `custom_rgbd_dataset.py`
   - Ensure synchronized augmentation between RGB and Depth

2. Refer to `Experiments/scripts/train_a3_rgbd.py` for the complete implementation

3. Expected improvements over A.3 old:
   - Better synchronized augmentation
   - More stable training
   - Potential mAP50-95 improvement

**Next Steps**:
- Compare results with A.1 (RGB only) and A.3 (old)
- Analyze failure cases
- Update `EXPERIMENT_GUIDE_V3.md` with results